# SBOL2Build

In [174]:
import sbol2
import tyto
import re
from Bio import Restriction
from Bio.Seq import Seq
from pydna.dseqrecord import Dseqrecord
from itertools import product
from typing import Dict, Iterable, List, Union, Optional, Tuple

In [175]:
# this document is used by all the next 
doc = sbol2.Document()
doc.addNamespace('http://SBOL2Build#', 'SBOL2Build')

# Restriction enzyme

## Target function in SBOL3

In [176]:
#target function

def ed_restriction_enzyme(name:str, **kwargs) -> sbol3.ExternallyDefined:
    """Creates an ExternallyDefined Restriction Enzyme Component from rebase.

    :param name: Name of the SBOL ExternallyDefined, used by PyDNA. Case sensitive, follow standard restriction enzyme nomenclature, i.e. 'BsaI'
    :param kwargs: Keyword arguments of any other ExternallyDefined attribute.
    :return: An ExternallyDefined object.
    """
    check_enzyme = Restriction.__dict__[name]
    definition=f'http://rebase.neb.com/rebase/enz/{name}.html' # TODO: replace with getting the URI from Enzyme when REBASE identifiers become available in biopython 1.8
    return sbol3.ExternallyDefined([sbol3.SBO_PROTEIN], definition=definition, name=name, **kwargs)

NameError: name 'sbol3' is not defined

## SBOL 2 implementation

In [ ]:
def rebase_restriction_enzyme(name:str, **kwargs) -> sbol2.ComponentDefinition:
    """Creates an ComponentDefinition Restriction Enzyme Component from rebase.

    :param name: Name of the SBOL ExternallyDefined, used by PyDNA. Case sensitive, follow standard restriction enzyme nomenclature, i.e. 'BsaI'
    :param kwargs: Keyword arguments of any other ComponentDefinition attribute.
    :return: A ComponentDefinition object.
    """
    check_enzyme = Restriction.__dict__[name]
    definition=f'http://rebase.neb.com/rebase/enz/{name}.html' # TODO: replace with getting the URI from Enzyme when REBASE identifiers become available in biopython 1.8
    cd = sbol2.ComponentDefinition(name)
    cd.types = sbol2.BIOPAX_PROTEIN
    cd.name = name
    cd.roles = []
    cd.wasDerivedFrom = definition
    cd.description = f'Restriction enzyme {name} from REBASE.'
    return cd



## Tests

In [ ]:
bsai = rebase_restriction_enzyme(name="BsaI")

In [ ]:
bsai.wasDerivedFrom
print(bsai.name)

BsaI


# DNA ComponentDefinition with Sequence

## Target function in SBOL3

In [ ]:
#target helper function
def dna_component_with_sequence(identity: str, sequence: str, **kwargs) -> Tuple[sbol3.Component, sbol3.Sequence]:
    """Creates a DNA Component and its Sequence.

    :param identity: The identity of the Component. The identity of Sequence is also identity with the suffix '_seq'.
    :param sequence: The DNA sequence of the Component encoded in IUPAC.
    :param kwargs: Keyword arguments of any other Component attribute.
    :return: A tuple of Component and Sequence.
    """
    comp_seq = sbol3.Sequence(f'{identity}_seq', elements=sequence, encoding=sbol3.IUPAC_DNA_ENCODING)
    dna_comp = sbol3.Component(identity, sbol3.SBO_DNA, sequences=[comp_seq], **kwargs)
    return dna_comp, comp_seq


## SBOL2 implementation

In [ ]:
def dna_componentdefinition_with_sequence2(identity: str, sequence: str, **kwargs) -> Tuple[sbol2.ComponentDefinition, sbol2.Sequence]:
    """Creates a DNA ComponentDefinition and its Sequence.

    :param identity: The identity of the Component. The identity of Sequence is also identity with the suffix '_seq'.
    :param sequence: The DNA sequence of the Component encoded in IUPAC.
    :param kwargs: Keyword arguments of any other Component attribute.
    :return: A tuple of ComponentDefinition and Sequence.
    """
    comp_seq = sbol2.Sequence(f'{identity}_seq', elements=sequence, encoding=sbol2.SBOL_ENCODING_IUPAC)
    dna_comp = sbol2.ComponentDefinition(identity, sbol2.BIOPAX_DNA, **kwargs)
    dna_comp.sequences = [comp_seq]

    return dna_comp, comp_seq

## Tests

# Target Part in Backbone from SBOL

In [ ]:
#target func

def part_in_backbone_from_sbol(identity: Union[str, None],  sbol_comp: sbol3.Component, part_location: List[int], part_roles:List[str], fusion_site_length:int, linear:bool=False, **kwargs) -> Tuple[sbol3.Component, sbol3.Sequence]:
    """Restructures a non-hierarchical plasmid Component to follow the part-in-backbone pattern following BP011.
    It overwrites the SBOL3 Component provided. 
    A part inserted into a backbone is represented by a Component that includes both the part insert 
    as a feature that is a SubComponent and the backbone as another SubComponent.
    For more information about BP011 visit https://github.com/SynBioDex/SBOL-examples/tree/main/SBOL/best-practices/BP011 

    :param identity: The identity of the Component, is its a String it build a new SBOL Component, if None it adds on top of the input. The identity of Sequence is also identity with the suffix '_seq'.
    :param sbol_comp: The SBOL3 Component that will be used to create the part in backbone Component and Sequence.
    :param part_location: List of 2 integers that indicates the start and the end of the unitary part. Note that the index of the first location is 1, as is typical practice in biology, rather than 0, as is typical practice in computer science.
    :param part_roles: List of strings that indicates the roles to add on the part.
    :param fusion_site_length: Integer of the length of the fusion sites (eg. BsaI fusion site lenght is 4, SapI fusion site lenght is 3)
    :param linear: Boolean than indicates if the backbone is linear, by default it is seted to Flase which means that it has a circular topology.    
    :param kwargs: Keyword arguments of any other Component attribute.
    :return: A tuple of Component and Sequence.
    """
    if len(part_location) != 2:
        raise ValueError('The part_location only accepts 2 int values in a list.')
    if len(sbol_comp.sequences)!=1:
        raise ValueError(f'The reactant needs to have precisely one sequence. The input reactant has {len(sbol_comp.sequences)} sequences')
    sequence = sbol_comp.sequences[0].lookup().elements
    if identity == None:
        part_in_backbone_component = sbol_comp 
        part_in_backbone_seq = sbol_comp.sequences[0]
    else:
        part_in_backbone_component, part_in_backbone_seq = dna_component_with_sequence(identity, sequence, **kwargs)
    part_in_backbone_component.roles.append(sbol3.SO_DOUBLE_STRANDED)
    for part_role in part_roles:  
        part_in_backbone_component.roles.append(part_role)  
    # creating part feature    
    part_location_comp = sbol3.Range(sequence=part_in_backbone_seq, start=part_location[0], end=part_location[1])
    #TODO: add the option of fusion sites to be of different lenghts
    insertion_site_location1 = sbol3.Range(sequence=part_in_backbone_seq, start=part_location[0], end=part_location[0]+fusion_site_length, order=1)
    insertion_site_location2 = sbol3.Range(sequence=part_in_backbone_seq, start=part_location[1]-fusion_site_length, end=part_location[1], order=3)
    part_sequence_feature = sbol3.SequenceFeature(locations=[part_location_comp], roles=part_roles)
    part_sequence_feature.roles.append(tyto.SO.engineered_insert)
    insertion_sites_feature = sbol3.SequenceFeature(locations=[insertion_site_location1, insertion_site_location2], roles=[tyto.SO.insertion_site])
    #TODO: infer topology from the input
    if linear:
        part_in_backbone_component.types.append(sbol3.SO_LINEAR)
        part_in_backbone_component.roles.append(sbol3.SO_ENGINEERED_REGION)
        # creating backbone feature
        open_backbone_location1 = sbol3.Range(sequence=part_in_backbone_seq, start=1, end=part_location[0]+fusion_site_length-1, order=1)
        open_backbone_location2 = sbol3.Range(sequence=part_in_backbone_seq, start=part_location[1]-fusion_site_length, end=len(sequence), order=3)
        open_backbone_feature = sbol3.SequenceFeature(locations=[open_backbone_location1, open_backbone_location2])
    else: 
        part_in_backbone_component.types.append(sbol3.SO_CIRCULAR)
        part_in_backbone_component.roles.append(tyto.SO.plasmid_vector)
        # creating backbone feature
        open_backbone_location1 = sbol3.Range(sequence=part_in_backbone_seq, start=1, end=part_location[0]+fusion_site_length-1, order=2)
        open_backbone_location2 = sbol3.Range(sequence=part_in_backbone_seq, start=part_location[1]-fusion_site_length, end=len(sequence), order=1)
        open_backbone_feature = sbol3.SequenceFeature(locations=[open_backbone_location1, open_backbone_location2])
    part_in_backbone_component.features.append(part_sequence_feature)
    part_in_backbone_component.features.append(insertion_sites_feature)
    part_in_backbone_component.features.append(open_backbone_feature)
    backbone_dropout_meets = sbol3.Constraint(restriction='http://sbols.org/v3#meets', subject=part_sequence_feature, object=open_backbone_feature)
    part_in_backbone_component.constraints.append(backbone_dropout_meets)
    #TODO: Add a branch to create a component without overwriting the WHOLE input component
    #removing repeated types and roles
    part_in_backbone_component.types = set(part_in_backbone_component.types)
    part_in_backbone_component.roles = set(part_in_backbone_component.roles)
    return part_in_backbone_component, part_in_backbone_seq

NameError: name 'sbol3' is not defined

## SBOL2 Part In Backbone Implementation

In [ ]:
def part_in_backbone_from_sbol2(identity: Union[str, None],  sbol_comp: sbol2.ModuleDefinition, part_location: List[int], part_roles:List[str], fusion_site_length:int, linear:bool=False, **kwargs) -> Tuple[sbol2.ComponentDefinition, sbol2.Sequence]:
    if len(part_location) != 2:
        raise ValueError('The part_location only accepts 2 int values in a list.')
    if len(sbol_comp.sequences)!=1:
        raise ValueError(f'The reactant needs to have precisely one sequence. The input reactant has {len(sbol_comp.sequences)} sequences')
    sequence = doc.find(sbol_comp.sequences[0]).elements
    if identity == None:
        part_in_backbone_component = sbol_comp 
        part_in_backbone_seq = doc.find(sbol_comp.sequences[0]).elements
        part_in_backbone_component.sequences = [part_in_backbone_seq]
    else:
        part_in_backbone_component, part_in_backbone_seq = dna_componentdefinition_with_sequence2(identity, sequence, **kwargs)
    # double stranded
    part_in_backbone_component.addRole('http://identifiers.org/SO:0000985')
    for part_role in part_roles:  
        part_in_backbone_component.addRole(part_role)

    # creating part annotation    
    part_location_comp = sbol2.Range( start=part_location[0], end=part_location[1])
    insertion_site_location1 = sbol2.Range( uri="insertloc1", start=part_location[0], end=part_location[0]+fusion_site_length) #order 1
    insertion_site_location2 = sbol2.Range( uri="insertloc2", start=part_location[1]-fusion_site_length, end=part_location[1]) #order 3

    part_sequence_annotation = sbol2.SequenceAnnotation('part_sequence_annotation')
    part_sequence_annotation.roles = part_roles
    part_sequence_annotation.locations.add(part_location_comp)

    part_sequence_annotation.addRole(tyto.SO.engineered_insert)
    insertion_sites_annotation = sbol2.SequenceAnnotation('insertion_sites_annotation')

    insertion_sites_annotation.locations.add(insertion_site_location1)
    insertion_sites_annotation.locations.add(insertion_site_location2)
    
    insertion_sites_annotation.roles = [tyto.SO.insertion_site]
    if linear:
        part_in_backbone_component.addRole('http://identifiers.org/SO:0000987') #linear
        part_in_backbone_component.addRole('http://identifiers.org/SO:0000804') #engineered region
        # creating backbone feature
        open_backbone_location1 = sbol2.Range(start=1, end=part_location[0]+fusion_site_length-1) #order 1
        open_backbone_location2 = sbol2.Range(start=part_location[1]-fusion_site_length, end=len(sequence)) #order 3
        open_backbone_annotation = sbol2.SequenceAnnotation(locations=[open_backbone_location1, open_backbone_location2])
    else: 
        part_in_backbone_component.addRole('http://identifiers.or/SO:0000988') #circular
        part_in_backbone_component.addRole(tyto.SO.plasmid_vector)
        # creating backbone feature
        open_backbone_location1 = sbol2.Range( uri="backboneloc1", start=1, end=part_location[0]+fusion_site_length-1 ) #order 2
        open_backbone_location2 = sbol2.Range( uri="backboneloc2", start=part_location[1]-fusion_site_length, end=len(sequence)) #order 1
        open_backbone_annotation = sbol2.SequenceAnnotation('open_backbone_annotation')
        open_backbone_annotation.locations.add(open_backbone_location1)
        open_backbone_annotation.locations.add(open_backbone_location2)
        
    part_in_backbone_component.sequenceAnnotations.add(part_sequence_annotation)
    part_in_backbone_component.sequenceAnnotations.add(insertion_sites_annotation)
    part_in_backbone_component.sequenceAnnotations.add(open_backbone_annotation) 
    # use sequenceconstrait with precedes
    # backbone_dropout_meets = sbol3.Constraint(restriction='http://sbols.org/v3#meets', subject=part_sequence_annotation, object=open_backbone_annotation) #????
    backbone_dropout_meets = sbol2.sequenceconstraint.SequenceConstraint(uri='backbone_dropout_meets', restriction=sbol2.SBOL_RESTRICTION_PRECEDES) #might need to add uri as param 2
    backbone_dropout_meets.subject = part_sequence_annotation
    backbone_dropout_meets.object = open_backbone_annotation
    
    part_in_backbone_component.sequenceConstraints.add(backbone_dropout_meets)
    #TODO: Add a branch to create a component without overwriting the WHOLE input component
    #removing repeated types and roles
    part_in_backbone_component.types = set(part_in_backbone_component.types)
    part_in_backbone_component.roles = set(part_in_backbone_component.roles)
    return part_in_backbone_component, part_in_backbone_seq

In [ ]:
# benchling plasmid test
doc = sbol2.Document()
benchling_comp = sbol2.ComponentDefinition('benchling_comp', sbol2.BIOPAX_DNA)

benchSeq = sbol2.Sequence('benchSeq', 'tcattgccatacgaaattccggatgagcattcatcaggcgggcaagaatgtgaataaaggccggataaaacttgtgcttatttttctttacggtctttaaaaaggccgtaatatccagctgaacggtctggttataggtacattgagcaactgactgaaatgcctcaaaatgttctttacgatgccattgggatatatcaacggtggtatatccagtgatttttttctccattttagcttccttagctcctgaaaatctcgataactcaaaaaatacgcccggtagtgatcttatttcattatggtgaaagttggaacctcttacgtgcccgatcaactcgagtgccacctgacgtctaagaaaccattattatcatgacattaacctataaaaataggcgtatcacgaggcagaatttcagataaaaaaaatccttagctttcgctaaggatgatttctggaattcggtctcgGGAGtttacagctagctcagtcctaggtattatgctagcTACTCGAGaccctgcagtccggcaaaaaagggcaaggtgtcaccaccctgccctttttctttaaaaccgaaaagattacttcgcgttatgcaggcttcctcgctcactgactcgctgcgctcggtcgttcggctgcggcgagcggtatcagctcactcaaaggcggtaatacggttatccacagaatcaggggataacgcaggaaagaacatgtgagcaaaaggccagcaaaaggccaggaaccgtaaaaaggccgcgttgctggcgtttttccacaggctccgcccccctgacgagcatcacaaaaatcgacgctcaagtcagaggtggcgaaacccgacaggactataaagataccaggcgtttccccctggaagctccctcgtgcgctctcctgttccgaccctgccgcttaccggatacctgtccgcctttctcccttcgggaagcgtggcgctttctcatagctcacgctgtaggtatctcagttcggtgtaggtcgttcgctccaagctgggctgtgtgcacgaaccccccgttcagcccgaccgctgcgccttatccggtaactatcgtcttgagtccaacccggtaagacacgacttatcgccactggcagcagccactggtaacaggattagcagagcgaggtatgtaggcggtgctacagagttcttgaagtggtggcctaactacggctacactagaagaacagtatttggtatctgcgctctgctgaagccagttaccttcggaaaaagagttggtagctcttgatccggcaaacaaaccaccgctggtagcggtggtttttttgtttgcaagcagcagattacgcgcagaaaaaaaggatctcaagaagatcctttgatcttttctacggggtctgacgctcagtggaacgaaaactcacgttaagggattttggtcatgagattatcaaaaaggatcttcacctagatccttttaaattaaaaatgaagttttaaatcaatctaaagtatatatgagtaaacttggtctgacagctcgaggcttggattctcaccaataaaaaacgcccggcggcaaccgagcgttctgaacaaatccagatggagttctgaggtcattactggatctatcaacaggagtccaagcgagctcgatatcaaattacgccccgccctgccactcatcgcagtactgttgtaattcattaagcattctgccgacatggaagccatcacaaacggcatgatgaacctgaatcgccagcggcatcagcaccttgtcgccttgcgtataatatttgcccatggtgaaaacgggggcgaagaagttgtccatattggccacgtttaaatcaaaactggtgaaactcacccagggattggctgagacgaaaaacatattctcaataaaccctttagggaaataggccaggttttcaccgtaacacgccacatcttgcgaatatatgtgtagaaactgccggaaatcgtcgtggtattcactccagagcgatgaaaacgtttcagtttgctcatggaaaacggtgtaacaagggtgaacactatcccatatcaccagctcaccgtct', sbol2.SBOL_ENCODING_IUPAC)

doc.addComponentDefinition(benchling_comp)
doc.addSequence(benchSeq)
benchling_comp.sequences = [benchSeq] 

resultComponent, resultSequence = part_in_backbone_from_sbol2(identity="benchling_comp", sbol_comp=benchling_comp, part_location=[531, 602], part_roles=[], fusion_site_length=6)

doc2 = sbol2.Document()

doc2.addComponentDefinition(resultComponent)
doc2.addComponentDefinition(resultSequence)


In [ ]:
doc2.write('example1.xml')

'Invalid. sbol-11403:\x00 Strong Validation Error:\x00 The Component referenced by the subject property of a SequenceConstraint MUST be contained by the ComponentDefinition that contains the SequenceConstraint. \x00Reference: SBOL Version 2.3.0 Section 7.7.6 on page 36 :\x00 http://examples.org/ComponentDefinition/benchling_comp/backbone_dropout_meets/1\x00  Validation failed.'

In [ ]:
# Initialize SBOL2 Document
sbol2.Config.setOption('sbol_typed_uris', True)
doc = sbol2.Document()

# Create a ComponentDefinition with a DNA sequence
component_def = sbol2.ComponentDefinition('example_component', sbol2.BIOPAX_DNA)
doc.addComponentDefinition(component_def)

# Add a sequence to the ComponentDefinition
sequence = sbol2.Sequence('example_sequence', 'ATGCTGACTGCTAGCTGACTAGC', sbol2.SBOL_ENCODING_IUPAC)
doc.addSequence(sequence)
component_def.sequences = [sequence.identity]  # Associate the sequence with the ComponentDefinition

# Create a SequenceAnnotation
seq_annotation = sbol2.SequenceAnnotation('example_annotation')
range_location = sbol2.Range('new range', 1, 6)  # Create a Range object
seq_annotation.locations.add(range_location)  # Add it to the annotation
component_def.sequenceAnnotations.add(seq_annotation)

# Optionally add a role (e.g., promoter, CDS) to describe the feature
seq_annotation.roles = ['http://identifiers.org/SO:0000167']  # Promoter role from Sequence Ontology

# Save the SBOL2 document
doc.write('sequence_annotation_example.xml')

print('SequenceAnnotation created successfully!')


In [ ]:
component_def = sbol2.ComponentDefinition('example_component', sbol2.BIOPAX_DNA)
sequence = sbol2.Sequence('example_sequence', 'ATGCTGACTGCTAGCTGACTAGC', sbol2.SBOL_ENCODING_IUPAC)
component_def.sequences = [sequence.identity]  # Associate the sequence with the ComponentDefinition


anno_1 = sbol2.SequenceAnnotation('new_anno')
anno_2 = sbol2.SequenceAnnotation('new_anno_2')

range_location = sbol2.Range('new range', 1, 6)  # Create a Range object
anno_1.locations.add(range_location)  # Add it to the annotation)

component_def.sequenceAnnotations.add(anno_1)
component_def.sequenceAnnotations.add(anno_2)

print(sbol2.SBOL_RESTRICTION_PRECEDES)

# new_constraint = sbol2.SequenceConstraint(restriction=SBOL_RESTRICTION_PRECEDES, subject=anno_1, object=anno_2) #????
backbone_dropout_meets = sbol2.sequenceconstraint.SequenceConstraint (
    'backbone_dropout',
    'backbone_dropout_uri',
    anno_1,
    anno_2,
    sbol2.SBOL_RESTRICTION_PRECEDES
)

component_def.sequenceConstraints.add(backbone_dropout_meets)

sbol2.DNA

In [ ]:
#target func
class Assembly_plan_composite_in_backbone_single_enzyme():
    """Creates a Assembly Plan.
    :param name: Name of the assembly plan Component.
    :param parts_in_backbone: Parts in backbone to be assembled. 
    :param acceptor_backbone:  Backbone in which parts are inserted on the assembly. 
    :param restriction_enzymes: Restriction enzyme with correct name from Bio.Restriction as Externally Defined.
    :param document: SBOL Document where the assembly plan will be created.
    :param linear: Boolean to inform if the reactant is linear.
    :param circular: Boolean to inform if the reactant is circular.
    :param **kwargs: Keyword arguments of any other Component attribute for the assembled part.
    """

    def __init__(self, name: str, parts_in_backbone: List[sbol3.Component], acceptor_backbone: sbol3.Component, restriction_enzyme: Union[str,sbol3.ExternallyDefined], document:sbol3.Document):
        self.name = name
        self.parts_in_backbone = parts_in_backbone
        self.acceptor_backbone = acceptor_backbone
        self.restriction_enzyme = restriction_enzyme
        self.products = []
        self.extracted_parts = []
        self.document = document

        #create assembly plan
        self.assembly_plan_component = sbol3.Component(identity=f'{self.name}_assembly_plan', types=sbol3.SBO_FUNCTIONAL_ENTITY)
        self.document.add(self.assembly_plan_component)
        self.composites = []

    def run(self):
        self.assembly_plan_component.features.append(self.restriction_enzyme)
        #extract parts
        part_number = 1
        for part_in_backbone in self.parts_in_backbone:
            part_comp, part_seq = digestion(reactant=part_in_backbone,restriction_enzymes=[self.restriction_enzyme], assembly_plan=self.assembly_plan_component, name=f'part_{part_number}_{part_in_backbone.display_id}')
            self.document.add([part_comp, part_seq])
            self.extracted_parts.append(part_comp)
            part_number += 1
        #extract backbone (should be the same?)
        backbone_comp, backbone_seq = digestion(reactant=self.acceptor_backbone,restriction_enzymes=[self.restriction_enzyme], assembly_plan=self.assembly_plan_component,  name=f'part_{part_number}')
        self.document.add([backbone_comp, backbone_seq])
        self.extracted_parts.append(backbone_comp)
        
        #create composite part from extracted parts
        composites_list = ligation(reactants=self.extracted_parts, assembly_plan=self.assembly_plan_component)
        for composite in composites_list:
            composite[0].generated_by.append(self.assembly_plan_component) #
            self.composites.append(composite)
            self.products.append(composite)
            self.document.add(composite)

# Digestion

## Target function in SBOL3

In [ ]:
def digestion(reactant:sbol3.Component, restriction_enzymes:List[sbol3.ExternallyDefined], assembly_plan:sbol3.Component, **kwargs)-> Tuple[sbol3.Component, sbol3.Sequence]:
    """Digests a Component using the provided restriction enzymes and creates a product Component and a digestion Interaction.
    The product Component is assumed to be the insert for parts in backbone and the backbone for backbones.

    :param reactant: DNA to be digested as SBOL Component, usually a part_in_backbone. 
    :param restriction_enzymes: Restriction enzymes used  Externally Defined.
    :return: A tuple of Component and Interaction.
    """
    if sbol3.SBO_DNA not in reactant.types:
        raise TypeError(f'The reactant should has a DNA type. Types founded {reactant.types}.')
    if len(reactant.sequences)!=1:
        raise ValueError(f'The reactant needs to have precisely one sequence. The input reactant has {len(reactant.sequences)} sequences')
    participations=[]
    restriction_enzymes_pydna=[] 
    for re in restriction_enzymes:
        enzyme = Restriction.__dict__[re.name]
        restriction_enzymes_pydna.append(enzyme)
        modifier_participation = sbol3.Participation(roles=[sbol3.SBO_MODIFIER], participant=re)
        participations.append(modifier_participation)

    # Inform topology to PyDNA, if not found assuming linear. 
    if is_circular(reactant):
        circular=True
        linear=False
    else: 
        circular=False
        linear=True
        
    reactant_seq = reactant.sequences[0].lookup().elements
    # Dseqrecord is from PyDNA package with reactant sequence
    ds_reactant = Dseqrecord(reactant_seq, linear=linear, circular=circular)
    digested_reactant = ds_reactant.cut(restriction_enzymes_pydna)

    if len(digested_reactant)<2 or len(digested_reactant)>3:
        raise NotImplementedError(f'Not supported number of products. Found{len(digested_reactant)}')
    #TODO select them based on content rather than size.
    elif circular and len(digested_reactant)==2:
        part_extract, backbone = sorted(digested_reactant, key=len)
    elif linear and len(digested_reactant)==3:
        prefix, part_extract, suffix = digested_reactant
    else: raise NotImplementedError('The reactant has no valid topology type')
    
    # Extracting roles from features
    reactant_features_roles = []
    for f in reactant.features:
        for r in f.roles:
             reactant_features_roles.append(r)
    # if part
    if any(n==tyto.SO.engineered_insert for n in reactant_features_roles):
        # Compute the length of single strand sticky ends or fusion sites
        product_5_prime_ss_strand, product_5_prime_ss_end = part_extract.seq.five_prime_end()
        product_3_prime_ss_strand, product_3_prime_ss_end = part_extract.seq.three_prime_end()
    
        product_sequence = str(part_extract.seq)
        prod_component_definition, prod_seq = dna_component_with_sequence(identity=f'{reactant.name}_part_extract', sequence=product_sequence, **kwargs) #str(product_sequence))
        # add sticky ends features
        five_prime_fusion_site_location = sbol3.Range(sequence=product_sequence, start=1, end=len(product_5_prime_ss_end), order=1)
        three_prime_fusion_site_location = sbol3.Range(sequence=product_sequence, start=len(product_sequence)-len(product_3_prime_ss_end)+1, end=len(product_sequence), order=3)
        fusion_sites_feature = sbol3.SequenceFeature(locations=[five_prime_fusion_site_location, three_prime_fusion_site_location], roles=[tyto.SO.insertion_site])
        prod_component_definition.roles.append(tyto.SO.engineered_insert) 
        prod_component_definition.features.append(fusion_sites_feature)

    # if backbone
    elif any(n==tyto.SO.deletion for n in reactant_features_roles):
        # Compute the length of single strand sticky ends or fusion sites
        product_5_prime_ss_strand, product_5_prime_ss_end = backbone.seq.five_prime_end()
        product_3_prime_ss_strand, product_3_prime_ss_end = backbone.seq.three_prime_end()
        product_sequence = str(backbone.seq)
        prod_component_definition, prod_seq = dna_component_with_sequence(identity=f'{reactant.name}_backbone', sequence=product_sequence, **kwargs) #str(product_sequence))
        # add sticky ends features
        five_prime_fusion_site_location = sbol3.Range(sequence=product_sequence, start=1, end=len(product_5_prime_ss_end), order=1)
        three_prime_fusion_site_location = sbol3.Range(sequence=product_sequence, start=len(product_sequence)-len(product_3_prime_ss_end)+1, end=len(product_sequence), order=3)
        fusion_sites_feature = sbol3.SequenceFeature(locations=[five_prime_fusion_site_location, three_prime_fusion_site_location], roles=[tyto.SO.insertion_site])
        prod_component_definition.roles.append(tyto.SO.plasmid_vector)
        prod_component_definition.features.append(fusion_sites_feature)

    else: raise NotImplementedError('The reactant has no valid roles')

    #Add reference to part in backbone
    reactant_subcomponent = sbol3.SubComponent(reactant)
    prod_component_definition.features.append(reactant_subcomponent)
    # Create reactant Participation.
    react_subcomp = sbol3.SubComponent(reactant)
    assembly_plan.features.append(react_subcomp)
    reactant_participation = sbol3.Participation(roles=[sbol3.SBO_REACTANT], participant=react_subcomp)
    participations.append(reactant_participation)
    
    prod_subcomp = sbol3.SubComponent(prod_comp)
    assembly_plan.features.append(prod_subcomp)
    product_participation = sbol3.Participation(roles=[sbol3.SBO_PRODUCT], participant=prod_subcomp)
    participations.append(product_participation)
   
    # Make Interaction
    interaction = sbol3.Interaction(types=[tyto.SBO.cleavage], participations=participations)
    assembly_plan.interactions.append(interaction)
                    
    return prod_comp, prod_seq

NameError: name 'sbol3' is not defined

# SBOL2 implementation

## Is Ciruclar

In [ ]:
# helper function
def is_circular(obj: sbol2.ComponentDefinition) -> bool:
    """Check if an SBOL Component or Feature is circular.
    :param obj: design to be checked
    :return: true if circular
    """    
    return any(n==sbol2.SO_CIRCULAR for n in obj.types)

### Tests

In [ ]:
import unittest

cd1 = sbol2.ComponentDefinition("circular_cd")
cd1.types = [sbol2.SO_CIRCULAR]
assert is_circular(cd1), "Circular ComponentDefinition should return True"

cd2 = sbol2.ComponentDefinition("non_circular_cd")
cd2.types = [sbol2.BIOPAX_DNA]
assert not is_circular(cd2), "Non-circular ComponentDefinition should return False"

cd3 = sbol2.ComponentDefinition("empty_types")
assert not is_circular(cd3), "Empty types list should return False"

print("All tests passed!")


All tests passed!


## Part Digestion

In [ ]:
def part_digestion(reactant:sbol2.ModuleDefinition, restriction_enzymes:List[sbol2.ComponentDefinition], assembly_plan:sbol2.ModuleDefinition, document: sbol2.Document, **kwargs)-> Tuple[List[Tuple[sbol2.ComponentDefinition, sbol2.Sequence]], sbol2.ModuleDefinition]:
    """Digests a ModuleDefinition using the provided restriction enzymes and creates a product ComponentDefinition and a digestion Interaction.
    The product ComponentDefinition is assumed to be the insert for parts in backbone and the open backbone for backbones.

    :param reactant: DNA to be digested as SBOL ModuleDefinition, usually a part_in_backbone. 
    :param restriction_enzymes: Restriction enzymes used ComponentDefinition.
    :param assembly_plan: SBOL ModuleDefinition to contain the functional components, interactions, and participants
    :param document: SBOL2 document to be used to extract referenced objects.
    :return: A list of tuples of [ComponentDefinition, Sequence], and an assemblyplan ModuleDefinition.
    """
    # extract component definition from module
    reactant_displayId = reactant.functionalComponents[0].displayId
    reactant_def_URI = reactant.functionalComponents[0].definition
    reactant_component_definition = document.getComponentDefinition(reactant_def_URI)

    if sbol2.BIOPAX_DNA not in reactant_component_definition.types:
        raise TypeError(f'The reactant should has a DNA type. Types found {reactant.types}.')
    if len(reactant_component_definition.sequences)!=1:
        raise ValueError(f'The reactant needs to have precisely one sequence. The input reactant has {len(reactant.sequences)} sequences')
    participations=[]
    extracts_list=[]
    restriction_enzymes_pydna=[] 
    
    for re in restriction_enzymes:
        enzyme = Restriction.__dict__[re.name]
        restriction_enzymes_pydna.append(enzyme)

        enzyme_component = sbol2.FunctionalComponent(uri=f"{re.name}_enzyme")
        enzyme_component.definition = re
        enzyme_component.displayID = f"{re.name}_enzyme"
        enzyme_in_module = False

        for comp in assembly_plan.functionalComponents:
            if (comp.displayId == enzyme_component.displayID): 
                enzyme_component = comp
                enzyme_in_module = True

        if not enzyme_in_module:
            assembly_plan.functionalComponents.add(enzyme_component)
            
        modifier_participation = sbol2.Participation(uri='restriction')
        modifier_participation.participant = enzyme_component
        modifier_participation.roles = ['http://identifiers.org/SBO:0000019'] #change to old?
        participations.append(modifier_participation)

    # Inform topology to PyDNA, if not found assuming linear. 
    if is_circular(reactant_component_definition):
        circular=True
        linear=False
    else: 
        circular=False
        linear=True
        
    reactant_seq = reactant_component_definition.sequences[0]
    reactant_seq = document.getSequence(reactant_seq).elements
    # Dseqrecord is from PyDNA package with reactant sequence
    ds_reactant = Dseqrecord(reactant_seq, circular=circular)
    digested_reactant = ds_reactant.cut(restriction_enzymes_pydna) #TODO see if ds_reactant.cut is working, causing problems downstream


    if len(digested_reactant)<2 or len(digested_reactant)>3: 
        raise NotImplementedError(f'Not supported number of products. Found{len(digested_reactant)}')
    #TODO select them based on content rather than size.
    elif circular and len(digested_reactant)==2:
        part_extract, backbone = sorted(digested_reactant, key=len)
    elif linear and len(digested_reactant)==3:
        prefix, part_extract, suffix = digested_reactant
    else: raise NotImplementedError('The reactant has no valid topology type')

    # Compute the length of single strand sticky ends or fusion sites
    product_5_prime_ss_strand, product_5_prime_ss_end = part_extract.seq.five_prime_end()
    product_3_prime_ss_strand, product_3_prime_ss_end = part_extract.seq.three_prime_end()
    product_sequence = str(part_extract.seq)
    prod_component_definition, prod_seq = dna_componentdefinition_with_sequence2(identity=f'{reactant.functionalComponents[0].displayId}_extracted_part', sequence=product_sequence, **kwargs)
    prod_component_definition.wasDerivedFrom = reactant_component_definition.identity
    extracts_list.append((prod_component_definition, prod_seq))

    # five prime overhang
    five_prime_oh_definition = sbol2.ComponentDefinition(uri=f"{reactant_displayId}_five_prime_oh") # TODO: ensure circular type is preserved for sbh visualization
    five_prime_oh_definition.addRole('http://identifiers.org/so/SO:0001932')
    five_prime_oh_location = sbol2.Range(uri="five_prime_oh_location", start=1, end=len(product_5_prime_ss_end))
    five_prime_oh_component = sbol2.Component(uri=f"{reactant_displayId}_five_prime_oh_component")
    five_prime_oh_component.definition = five_prime_oh_definition
    five_prime_overhang_annotation = sbol2.SequenceAnnotation(uri="five_prime_overhang")
    five_prime_overhang_annotation.locations.add(five_prime_oh_location)

    # extracted part => point straight to part from sbolcanvas
    part_location = sbol2.Range(uri=f"{reactant_displayId}_part_location", start=len(product_5_prime_ss_end)+1, end=len(product_sequence)-len(product_3_prime_ss_end))
    part_extract_annotation = sbol2.SequenceAnnotation(uri=f"{reactant_displayId}_part")
    part_extract_annotation.locations.add(part_location)

    # three prime overhang
    three_prime_oh_definition = sbol2.ComponentDefinition(uri=f"{reactant_displayId}_three_prime_oh")
    three_prime_oh_definition.addRole('http://identifiers.org/so/SO:0001933')
    three_prime_oh_location = sbol2.Range(uri="three_prime_oh_location", start=len(product_sequence)-len(product_3_prime_ss_end)+1, end=len(product_sequence)) 
    three_prime_oh_component = sbol2.Component(uri=f"{reactant_displayId}_three_prime_oh_component")
    three_prime_oh_component.definition = three_prime_oh_definition
    three_prime_overhang_annotation = sbol2.SequenceAnnotation(uri="three_prime_overhang")
    three_prime_overhang_annotation.locations.add(three_prime_oh_location)
    
    # three_prime_overhang_annotation.addRole(tyto.SO.insertion_site) #TODO: do we need this, or change to three_prime_overhang_annotation.addRole('http://identifiers.org/SO:0000366')?

    anno_component_dict = {}

    prod_component_definition.components = [five_prime_oh_component, three_prime_oh_component]
    three_prime_overhang_annotation.component = three_prime_oh_component
    five_prime_overhang_annotation.component = five_prime_oh_component

    original_part_def_URI = ""

    #enccode ontologies of overhangs
    for definition in document.componentDefinitions:
        for seqURI in definition.sequences:
            seq = document.getSequence(seqURI)
            if seq.elements.lower() == Seq(product_3_prime_ss_end).reverse_complement():
                three_prime_oh_definition.wasDerivedFrom = definition.identity
                three_prime_sequence = sbol2.Sequence(uri=f"{three_prime_oh_definition.displayId}_sequence", elements= seq.elements)
                three_prime_sequence.wasDerivedFrom = seq.identity
                three_prime_oh_definition.sequences.append(three_prime_sequence)
                three_prime_oh_definition.types.append("http://identifiers.org/SO:0000984") # single-stranded for overhangs
                
                extracts_list.append((three_prime_oh_definition, three_prime_sequence))
                extracts_list.append((definition, seq)) #add scars to list

            elif seq.elements.lower() == product_sequence[4:-4]: 
                original_part_def_URI = definition.identity
                extracts_list.append((definition, seq))

            elif seq.elements.lower() == product_5_prime_ss_end:
                five_prime_oh_definition.wasDerivedFrom = definition.identity
                five_prime_sequence = sbol2.Sequence(uri=f"{five_prime_oh_definition.displayId}_sequence", elements= seq.elements)
                five_prime_sequence.wasDerivedFrom = seq.identity
                five_prime_oh_definition.sequences.append(five_prime_sequence)
                five_prime_oh_definition.types.append("http://identifiers.org/SO:0000984") # single-stranded for overhangs

                extracts_list.append((five_prime_oh_definition, five_prime_sequence))
                extracts_list.append((definition, seq))

    #find + add original component to product def & annotation
    for comp in reactant_component_definition.components:        
        if comp.definition == original_part_def_URI:
            prod_component_definition.components.add(comp)
            part_extract_annotation.component = comp

    prod_component_definition.sequenceAnnotations.add(three_prime_overhang_annotation)
    prod_component_definition.sequenceAnnotations.add(five_prime_overhang_annotation)
    prod_component_definition.sequenceAnnotations.add(part_extract_annotation)
    prod_component_definition.addRole(tyto.SO.engineered_insert) 

    #Add reference to part in backbone
    reactant_component = sbol2.FunctionalComponent(uri=f"{reactant_displayId}_reactant_component")
    reactant_component.definition = reactant_component_definition
    assembly_plan.functionalComponents.add(reactant_component)

    # Create reactant Participation.
    reaction_component = sbol2.FunctionalComponent(uri=f"{reactant_displayId}_reaction_component")
    reaction_component.definition = reactant_component_definition #ask gonzalo (do we need reaction component in addition to reactant and participations? see target func)
    assembly_plan.functionalComponents.add(reaction_component)

    reactant_participation = sbol2.Participation(uri=f"{reactant_displayId}_reactant_participation")
    reactant_participation.participant = reaction_component
    reactant_participation.roles = [sbol2.SBO_REACTANT]
    participations.append(reactant_participation)
    
    prod_component = sbol2.FunctionalComponent(uri=f"{reactant_displayId}_product_component")
    prod_component.definition = prod_component_definition
    assembly_plan.functionalComponents.add(prod_component)

    product_participation = sbol2.Participation(uri=f"{reactant_displayId}_product_participation")
    product_participation.participant = prod_component
    product_participation.roles = [sbol2.SBO_PRODUCT]
    participations.append(product_participation)
   
    # Make Interaction
    interaction = sbol2.Interaction(uri=f"{reactant_displayId}_asssembly_plan_interaction", interaction_type=tyto.SBO.cleavage)
    interaction.participations = participations
    assembly_plan.interactions.add(interaction)
                    
    return extracts_list, assembly_plan

### Tests

In [ ]:
pro_in_bb_doc = sbol2.Document()
pro_in_bb_doc.read('Test/pro_in_bb.xml')
md = pro_in_bb_doc.getModuleDefinition("https://sbolcanvas.org/module1")
assembly_plan = sbol2.ModuleDefinition('insert_assembly_plan')

extracts_tuple_list, assembly_plan = part_digestion(md, [bsai], assembly_plan, pro_in_bb_doc)
part_extract, seq = extracts_tuple_list[0]

new_doc = sbol2.Document()
for extract, sequence in extracts_tuple_list:
    print(extract.identity)
    print(sequence.identity)
    new_doc.add(extract)
    new_doc.add(sequence)
new_doc.add(assembly_plan)
new_doc.write('new_promoter_test.xml')

http://examples.org/ComponentDefinition/UJHDBOTD_70_extracted_part/1
http://examples.org/Sequence/UJHDBOTD_70_extracted_part_seq/1
http://examples.org/ComponentDefinition/UJHDBOTD_70_three_prime_oh/1
http://examples.org/Sequence/UJHDBOTD_70_three_prime_oh_sequence/1
https://sbolcanvas.org/Scar_B/1
https://sbolcanvas.org/Scar_B_sequence
https://sbolcanvas.org/J23101/1
https://sbolcanvas.org/J23101_sequence
http://examples.org/ComponentDefinition/UJHDBOTD_70_five_prime_oh/1
http://examples.org/Sequence/UJHDBOTD_70_five_prime_oh_sequence/1
https://sbolcanvas.org/Scar_A/1
https://sbolcanvas.org/Scar_A_sequence


'Valid.'

## Backbone Digestion

In [ ]:
def backbone_digestion(reactant:sbol2.ModuleDefinition, restriction_enzymes:List[sbol2.ComponentDefinition], assembly_plan:sbol2.ModuleDefinition, document: sbol2.Document, **kwargs)-> Tuple[List[Tuple[sbol2.ComponentDefinition, sbol2.Sequence]], sbol2.ModuleDefinition]:
    """Digests a ModuleDefinition using the provided restriction enzymes and creates a product ComponentDefinition and a digestion Interaction.
    The product ComponentDefinition is assumed to be the insert for parts in backbone and the open backbone for backbones.

    :param reactant: DNA to be digested as SBOL ModuleDefinition, usually a part_in_backbone. 
    :param restriction_enzymes: Restriction enzymes used ComponentDefinition.
    :param assembly_plan: SBOL ModuleDefinition to contain the functional components, interactions, and participants
    :param document: SBOL2 document to be used to extract referenced objects.
    :return: A tuple of ComponentDefinition, Sequence, and ModuleDefinition.
    """
    # extract component definition from module
    reactant_displayId = reactant.functionalComponents[0].displayId
    reactant_def_URI = reactant.functionalComponents[0].definition
    reactant_component_definition = document.getComponentDefinition(reactant_def_URI)

    if sbol2.BIOPAX_DNA not in reactant_component_definition.types:
        raise TypeError(f'The reactant should has a DNA type. Types founded {reactant.types}.')
    if len(reactant_component_definition.sequences)!=1: # TODO review if true for MD, maybe for MD it will be 5
        raise ValueError(f'The reactant needs to have precisely one sequence. The input reactant has {len(reactant.sequences)} sequences')
    participations=[]
    extracts_list=[]
    restriction_enzymes_pydna=[] 
    
    for re in restriction_enzymes:
        enzyme = Restriction.__dict__[re.name]
        restriction_enzymes_pydna.append(enzyme)

        enzyme_component = sbol2.FunctionalComponent(uri=f"{re.name}_enzyme")
        enzyme_component.definition = re
        enzyme_component.displayID = f"{re.name}_enzyme"
        enzyme_in_module = False

        for comp in assembly_plan.functionalComponents:
            if (comp.displayId == enzyme_component.displayID):
                enzyme_component = comp
                enzyme_in_module = True

        if not enzyme_in_module:
            assembly_plan.functionalComponents.add(enzyme_component)
            
        modifier_participation = sbol2.Participation(uri='restriction')
        modifier_participation.participant = enzyme_component
        modifier_participation.roles = ['http://identifiers.org/SBO:0000019']
        participations.append(modifier_participation)

    # Inform topology to PyDNA, if not found assuming linear. 
    if is_circular(reactant_component_definition):
        circular=True
        linear=False
    else: 
        circular=False
        linear=True
        
    reactant_seq = reactant_component_definition.sequences[0]
    reactant_seq = document.getSequence(reactant_seq).elements
    # Dseqrecord is from PyDNA package with reactant sequence
    ds_reactant = Dseqrecord(reactant_seq, circular=circular)
    digested_reactant = ds_reactant.cut(restriction_enzymes_pydna) #TODO see if ds_reactant.cut is working, causing problems downstream


    if len(digested_reactant)<2 or len(digested_reactant)>3: 
        raise NotImplementedError(f'Not supported number of products. Found{len(digested_reactant)}')
    #TODO select them based on content rather than size.
    elif circular and len(digested_reactant)==2:
        part_extract, backbone = sorted(digested_reactant, key=len)
    elif linear and len(digested_reactant)==3:
        prefix, part_extract, suffix = digested_reactant
    else: raise NotImplementedError('The reactant has no valid topology type')

    # Compute the length of single strand sticky ends or fusion sites
    product_5_prime_ss_strand, product_5_prime_ss_end = backbone.seq.five_prime_end()
    product_3_prime_ss_strand, product_3_prime_ss_end = backbone.seq.three_prime_end()
    product_sequence = str(backbone.seq)
    prod_backbone_definition, prod_seq = dna_componentdefinition_with_sequence2(identity=f'{reactant_component_definition.displayId}_extracted_backbone', sequence=product_sequence, **kwargs)
    prod_backbone_definition.wasDerivedFrom = reactant_component_definition.identity
    extracts_list.append((prod_backbone_definition, prod_seq))

    # five prime overhang
    five_prime_oh_definition = sbol2.ComponentDefinition(uri=f"{reactant_displayId}_five_prime_oh") # TODO: ensure circular type is preserved for sbh visualization
    five_prime_oh_definition.addRole('http://identifiers.org/so/SO:0001932')
    five_prime_oh_location = sbol2.Range(uri="five_prime_oh_location", start=1, end=len(product_5_prime_ss_end))
    five_prime_oh_component = sbol2.Component(uri=f"{reactant_displayId}_five_prime_oh_component")
    five_prime_oh_component.definition = five_prime_oh_definition
    five_prime_overhang_annotation = sbol2.SequenceAnnotation(uri="five_prime_overhang")
    five_prime_overhang_annotation.locations.add(five_prime_oh_location)

    # extracted backbone => point straight to backbone from sbolcanvas
    backbone_location = sbol2.Range(uri=f"{reactant_displayId}_backbone_location", start=len(product_5_prime_ss_end)+1, end=len(product_sequence)-len(product_3_prime_ss_end))
    backbone_extract_annotation = sbol2.SequenceAnnotation(uri=f"{reactant_displayId}_backbone")
    backbone_extract_annotation.locations.add(backbone_location)
    
    # three prime overhang
    three_prime_oh_definition = sbol2.ComponentDefinition(uri=f"{reactant_displayId}_three_prime_oh")
    three_prime_oh_definition.addRole('http://identifiers.org/so/SO:0001933')
    three_prime_oh_location = sbol2.Range(uri="three_prime_oh_location", start=len(product_sequence)-len(product_3_prime_ss_end)+1, end=len(product_sequence)) 
    three_prime_oh_component = sbol2.Component(uri=f"{reactant_displayId}_three_prime_oh_component")
    three_prime_oh_component.definition = three_prime_oh_definition
    three_prime_overhang_annotation = sbol2.SequenceAnnotation(uri="three_prime_overhang")
    three_prime_overhang_annotation.locations.add(three_prime_oh_location)

    anno_component_dict = {}

    prod_backbone_definition.components = [five_prime_oh_component, three_prime_oh_component]
    three_prime_overhang_annotation.component = three_prime_oh_component
    five_prime_overhang_annotation.component = five_prime_oh_component

    # check these lines
    original_backbone_def_URI = ""

    #enccode ontologies of overhangs
    for definition in document.componentDefinitions:
        for seqURI in definition.sequences:
            seq = document.getSequence(seqURI)
            if seq.elements.lower() == Seq(product_3_prime_ss_end).reverse_complement():
                three_prime_oh_definition.wasDerivedFrom = definition.identity
                three_prime_sequence = sbol2.Sequence(uri=f"{three_prime_oh_definition.displayId}_sequence", elements= seq.elements)
                three_prime_sequence.wasDerivedFrom = seq.identity
                three_prime_oh_definition.sequences.append(three_prime_sequence)
                three_prime_oh_definition.types.append("http://identifiers.org/SO:0000984") # single-stranded for overhangs
                
                extracts_list.append((three_prime_oh_definition, three_prime_sequence))
                extracts_list.append((definition, seq)) #add scars to list

            elif seq.elements.lower() == product_sequence[4:-4]: 
                original_backbone_def_URI = definition.identity
                extracts_list.append((definition, seq))

            elif seq.elements.lower() == product_5_prime_ss_end:
                five_prime_oh_definition.wasDerivedFrom = definition.identity
                five_prime_sequence = sbol2.Sequence(uri=f"{five_prime_oh_definition.displayId}_sequence", elements= seq.elements)
                five_prime_sequence.wasDerivedFrom = seq.identity
                five_prime_oh_definition.sequences.append(five_prime_sequence)
                five_prime_oh_definition.types.append("http://identifiers.org/SO:0000984") # single-stranded for overhangs

                extracts_list.append((five_prime_oh_definition, five_prime_sequence))
                extracts_list.append((definition, seq))

    #find + add original component to product def & annotation
    for comp in reactant_component_definition.components:        
        if comp.definition == original_backbone_def_URI:
            prod_backbone_definition.components.add(comp)
            backbone_extract_annotation.component = comp

    prod_backbone_definition.sequenceAnnotations.add(three_prime_overhang_annotation)
    prod_backbone_definition.sequenceAnnotations.add(five_prime_overhang_annotation)
    prod_backbone_definition.sequenceAnnotations.add(backbone_extract_annotation)
    prod_backbone_definition.addRole(tyto.SO.plasmid_vector) 

    #Add reference to part in backbone
    reactant_component = sbol2.FunctionalComponent(uri=f'{reactant_component_definition.displayId}_backbone_reactant_component')
    reactant_component.definition = reactant_component_definition
    assembly_plan.functionalComponents.add(reactant_component)

    # Create reactant Participation.
    reaction_component = sbol2.FunctionalComponent(uri=f'{reactant_component_definition.displayId}_backbone_reaction_component')
    reaction_component.definition = reactant_component_definition
    assembly_plan.functionalComponents.add(reaction_component)

    reactant_participation = sbol2.Participation(uri=f'{reactant_component_definition.displayId}_backbone_reactant_participation')
    reactant_participation.participant = reaction_component
    reactant_participation.roles = [sbol2.SBO_REACTANT]
    participations.append(reactant_participation)
    
    prod_component = sbol2.FunctionalComponent(uri=f'{reactant_component_definition.displayId}_backbone_product_component')
    prod_component.definition = prod_backbone_definition
    assembly_plan.functionalComponents.add(prod_component)

    product_participation = sbol2.Participation(uri=f'{reactant_component_definition.displayId}_backbone_product_participation')
    product_participation.participant = prod_component
    product_participation.roles = [sbol2.SBO_PRODUCT]
    participations.append(product_participation)
   
    # Make Interaction
    interaction = sbol2.Interaction(uri="asssembly_plan_interaction", interaction_type=tyto.SBO.cleavage)
    interaction.participations = participations
    assembly_plan.interactions.add(interaction)
                    
    return extracts_list, assembly_plan

### Tests

In [ ]:
bb_doc = sbol2.Document()
bb_doc.read('Test/backbone.xml')

md = bb_doc.getModuleDefinition("https://sbolcanvas.org/module1")
assembly_plan = sbol2.ModuleDefinition('bb_assembly_plan')

extracts_tuple_list, assembly_plan = backbone_digestion(md, [bsai], assembly_plan, bb_doc)
part_extract, seq = extracts_tuple_list[0]

new_doc = sbol2.Document()
for extract, sequence in extracts_tuple_list:
    print(extract.identity)
    print(sequence.identity)
    new_doc.add(extract)
    new_doc.add(sequence)
new_doc.add(assembly_plan)
print(f"should include role: {tyto.SO.plasmid_vector}")
new_doc.write('new_bb_test.xml')

http://examples.org/ComponentDefinition/UJHDBOTD_extracted_backbone/1
http://examples.org/Sequence/UJHDBOTD_extracted_backbone_seq/1
http://examples.org/ComponentDefinition/UJHDBOTD_159_five_prime_oh/1
http://examples.org/Sequence/UJHDBOTD_159_five_prime_oh_sequence/1
https://sbolcanvas.org/Scar_F/1
https://sbolcanvas.org/Scar_F_sequence
https://sbolcanvas.org/Cir_qxow/1
https://sbolcanvas.org/Cir_qxow_sequence
http://examples.org/ComponentDefinition/UJHDBOTD_159_three_prime_oh/1
http://examples.org/Sequence/UJHDBOTD_159_three_prime_oh_sequence/1
https://sbolcanvas.org/Scar_A/1
https://sbolcanvas.org/Scar_A_sequence
should include role: https://identifiers.org/SO:0000755


'Valid.'

# Ligation

In [ ]:
#target function
def ligation(reactants:List[sbol3.Component], assembly_plan:sbol3.Component)-> List[Tuple[sbol3.Component, sbol3.Sequence]]:
    """Ligates Components using base complementarity and creates a product Component and a ligation Interaction.

    :param reactant: DNA to be ligated as SBOL Component. 
    :return: A tuple of Component and Interaction.
    """
    # Create a dictionary that maps each first and last 4 letters to a list of strings that have those letters.
    reactant_parts = []
    fusion_sites_set = set()
    for reactant in reactants:
        fusion_site_3prime_length = reactant.features[0].locations[0].end - reactant.features[0].locations[0].start
        fusion_site_5prime_length = reactant.features[0].locations[1].end - reactant.features[0].locations[1].start
        if fusion_site_3prime_length == fusion_site_5prime_length:
            fusion_site_length = fusion_site_3prime_length + 1 # if the fusion site is 4 bp long, the start will be 1 and end 4, 4-1 = 3, so we add 1 to get 4.
            fusion_sites_set.add(fusion_site_length)
            if len(fusion_sites_set) > 1:
                raise ValueError(f'Fusion sites of different length within different parts. Check {reactant.identity} ')
        else:
            raise ValueError(f'Fusion sites of different length within the same part. Check {reactant.identity}')
        if tyto.SO.plasmid_vector in reactant.roles:
            reactant_parts.append(reactant)
        elif tyto.SO.engineered_insert in reactant.roles:
            reactant_parts.append(reactant)
        else:
            raise ValueError(f'Part {reactant.identity} does not have a valid role')
    # remove the backbones if any from the reactants, to create the composite
    groups = {}
    for reactant in reactant_parts:
        first_four_letters = reactant.sequences[0].lookup().elements[:fusion_site_length].lower()
        last_four_letters = reactant.sequences[0].lookup().elements[-fusion_site_length:].lower()
        part_syntax = f'{first_four_letters}_{last_four_letters}'
        if part_syntax not in groups:
            groups[part_syntax] = []
            groups[part_syntax].append(reactant)
        else: groups[part_syntax].append(reactant)
    # groups is a dictionary of lists of parts that have the same first and last 4 letters
    # list_of_combinations_per_assembly is a list of tuples of parts that can be ligated together
    list_of_parts_per_combination = list(product(*groups.values())) #cartesian product
    # create list_of_composites_per_assembly from list_of_combinations_per_assembly
    list_of_composites_per_assembly = []
    for combination in list_of_parts_per_combination:
        list_of_parts_per_composite = [combination[0]] 
        insert_sequence = combination[0].sequences[0].lookup().elements
        remaining_parts = list(combination[1:])
        it = 1
        while remaining_parts:   
            remaining_parts_before = len(remaining_parts) 
            for part in remaining_parts:
                # match insert sequence 5' to part 3'
                if part.sequences[0].lookup().elements[:fusion_site_length].lower() == insert_sequence[-fusion_site_length:].lower():
                    insert_sequence = insert_sequence[:-fusion_site_length] + part.sequences[0].lookup().elements
                    list_of_parts_per_composite.append(part)
                    remaining_parts.remove(part)
                # match insert sequence 3' to part 5'
                elif part.sequences[0].lookup().elements[-fusion_site_length:].lower() == insert_sequence[:fusion_site_length].lower():
                    insert_sequence =  part.sequences[0].lookup().elements + insert_sequence[fusion_site_length:]
                    list_of_parts_per_composite.insert(0, part)
                    remaining_parts.remove(part)
                remaining_parts_after = len(remaining_parts)
            
            if remaining_parts_before == remaining_parts_after:
                it += 1
            if it > 5: #5 was chosen arbitrarily to avoid infinite loops
                print(groups)
                raise ValueError('No match found, check the parts and their fusion sites')
        list_of_composites_per_assembly.append(list_of_parts_per_composite)

    # transform list_of_parts_per_assembly into list of composites
    products_list = []
    participations = []
    composite_number = 1
    for composite in list_of_composites_per_assembly: # a composite of the form [A,B,C]
        # calculate sequence
        composite_sequence_str = ""
        composite_name = ""
        #part_subcomponents = []
        part_extract_subcomponents = []
        for part_extract in composite:
            composite_sequence_str = composite_sequence_str + part_extract.sequences[0].lookup().elements[:-fusion_site_length] #needs a version for linear
            # create participations
            part_extract_subcomponent = sbol3.SubComponent(part_extract) # LocalSubComponent??
            part_extract_subcomponents.append(part_extract_subcomponent)
            composite_name = composite_name +'_'+ part_extract.name
        # create dna componente and sequence
        composite_component, composite_seq = dna_component_with_sequence(f'composite_{composite_number}{composite_name}', composite_sequence_str) # **kwarads use in future?
        composite_component.name = f'composite_{composite_number}{composite_name}'
        composite_component.roles.append(sbol3.SO_ENGINEERED_REGION)
        composite_component.features = part_extract_subcomponents
        for i in range(len(composite_component.features )-1):
            composite_component.constraints = [sbol3.Constraint(restriction='http://sbols.org/v3#meets', subject=composite_component.features[i], object=composite_component.features[i+1])]
        products_list.append([composite_component, composite_seq])
        composite_number += 1
    return products_list

### Alphabetical Suffix Helper Function (for scars)

In [ ]:
def number_to_suffix(n):
    suffix = ""
    while n > 0:
        n -= 1  # Adjust to make A = 0, B = 1, ..., Z = 25
        remainder = n % 26  # Get the current character
        suffix = chr(ord('A') + remainder) + suffix  # Build the suffix
        n = n // 26  # Move to the next higher place value
    return suffix


## SBOL2 implementation

In [ ]:
def ligation2(reactants:List[sbol2.ComponentDefinition], assembly_plan: sbol2.ModuleDefinition, document: sbol2.Document)->List[Tuple[sbol2.ComponentDefinition, sbol2.Sequence]]:
    """Ligates Components using base complementarity and creates a product Component and a ligation Interaction.

    :param reactant: DNA to be ligated as SBOL ModuleDefinition. 
    :param assembly_plan: SBOL ModuleDefinition to contain the functional components, interactions, and participants
    :param document: SBOL2 document containing all reactant componentdefinitions.
    :return: A tuple of ComponentDefinition and Sequence.
    """

    # Create a dictionary that maps each first and last 4 letters to a list of strings that have those letters.
    reactant_parts = []
    fusion_sites_set = set()
    for reactant in reactants:
        fusion_site_3prime_length = reactant.sequenceAnnotations[0].locations[0].end - reactant.sequenceAnnotations[0].locations[0].start
        fusion_site_5prime_length = reactant.sequenceAnnotations[1].locations[0].end - reactant.sequenceAnnotations[1].locations[0].start
        if fusion_site_3prime_length == fusion_site_5prime_length:
            fusion_site_length = fusion_site_3prime_length + 1 # if the fusion site is 4 bp long, the start will be 1 and end 4, 4-1 = 3, so we add 1 to get 4.
            fusion_sites_set.add(fusion_site_length)
            if len(fusion_sites_set) > 1:
                raise ValueError(f'Fusion sites of different length within different parts. Check {reactant.identity} ')
        else:
            raise ValueError(f'Fusion sites of different length within the same part. Check {reactant.identity}')
        if tyto.SO.plasmid_vector in reactant.roles:
            reactant_parts.append(reactant)
        elif tyto.SO.engineered_insert in reactant.roles:
            reactant_parts.append(reactant)
        else:
            raise ValueError(f'Part {reactant.identity} does not have a valid role')
        
    # remove the backbones if any from the reactants, to create the composite
    groups = {}
    for reactant in reactant_parts:
        reactant_seq = reactant.sequences[0]
        first_four_letters = document.getSequence(reactant_seq).elements[:fusion_site_length].lower()
        last_four_letters = document.getSequence(reactant_seq).elements[-fusion_site_length:].lower()
        part_syntax = f'{first_four_letters}_{last_four_letters}'
        if part_syntax not in groups:
            groups[part_syntax] = []
            groups[part_syntax].append(reactant)
        else: groups[part_syntax].append(reactant)
    # groups is a dictionary of lists of parts that have the same first and last 4 letters
    # list_of_combinations_per_assembly is a list of tuples of parts that can be ligated together
    list_of_parts_per_combination = list(product(*groups.values())) #cartesian product
    # create list_of_composites_per_assembly from list_of_combinations_per_assembly
    list_of_composites_per_assembly = []
    for combination in list_of_parts_per_combination:
        list_of_parts_per_composite = [combination[0]] 
        insert_sequence_uri = combination[0].sequences[0]
        insert_sequence = document.getSequence(insert_sequence_uri).elements
        remaining_parts = list(combination[1:])
        it = 1
        while remaining_parts:   
            remaining_parts_before = len(remaining_parts) 
            for part in remaining_parts:
                # match insert sequence 5' to part 3'
                part_sequence_uri = part.sequences[0]
                if document.getSequence(part_sequence_uri).elements[:fusion_site_length].lower() == insert_sequence[-fusion_site_length:].lower():
                    insert_sequence = insert_sequence[:-fusion_site_length] + document.getSequence(part_sequence_uri).elements
                    list_of_parts_per_composite.append(part) #add sequence annotation here, index based on insert_sequence
                    remaining_parts.remove(part)
                # match insert sequence 3' to part 5'
                elif document.getSequence(part_sequence_uri).elements[-fusion_site_length:].lower() == insert_sequence[:fusion_site_length].lower():
                    insert_sequence = document.getSequence(part_sequence_uri).elements + insert_sequence[fusion_site_length:]
                    list_of_parts_per_composite.insert(0, part)
                    remaining_parts.remove(part)
                remaining_parts_after = len(remaining_parts)
            
            if remaining_parts_before == remaining_parts_after:
                it += 1
            if it > 5: #5 was chosen arbitrarily to avoid infinite loops
                print(groups)
                raise ValueError('No match found, check the parts and their fusion sites')
        list_of_composites_per_assembly.append(list_of_parts_per_composite)

    # transform list_of_parts_per_assembly into list of composites
    products_list = []
    participations = []
    composite_number = 1
    print(f"list composites per assembly {list_of_composites_per_assembly}")
    # TODO: use componentinstances to append "subcomponents" to each definition that is a composite component. all composites share the "subcomponents"
    for composite in list_of_composites_per_assembly: # a composite of the form [A,B,C]
        # calculate sequence
        composite_sequence_str = ""
        composite_name = ""
        prev_three_prime = composite[len(composite)-1].components[1].definition # componentdefinitionuri
        print("3p:", prev_three_prime)
        scar_index = 1
        anno_list = []

        part_extract_definitions = []
        for part_extract in composite:
            part_extract_sequence_uri = part_extract.sequences[0]
            part_extract_sequence = document.getSequence(part_extract_sequence_uri).elements
            # TODO create participations & annotations
            # print(f"part_extract.components: {[component.definition for component in part_extract.components]}")
            temp_extract_components = []
            for comp in part_extract.components:
                if  "http://identifiers.org/so/SO:0001932" in document.getComponentDefinition(comp.definition).roles: #five prime
                    scar_definition = sbol2.ComponentDefinition(uri=f"Scar_{number_to_suffix(scar_index)}")
                    scar_definition.wasDerivedFrom = [comp.definition, prev_three_prime] # need case for if prev_three_prime is empty (maybe leave overhang and check for sticky end match after sequence is assembled)
                    scar_definition.roles = ["http://identifiers.org/so/SO:0001953"]
                    temp_extract_components.append(scar_definition.identity)
                    document.add(scar_definition)

                    scar_location = sbol2.Range(uri=f"Scar_{number_to_suffix(scar_index)}_location", start= len(composite_sequence_str)+1, end=len(composite_sequence_str)+fusion_site_length)
                    scar_anno = sbol2.SequenceAnnotation(uri=f"Scar_{number_to_suffix(scar_index)}_annotation")
                    scar_anno.locations.add(scar_location)
                    anno_list.append(scar_anno)
                    scar_index += 1
                elif "http://identifiers.org/so/SO:0001933" in document.getComponentDefinition(comp.definition).roles: #three prime
                    prev_three_prime = comp.definition
                else:
                    temp_extract_components.append(comp.definition)
                    comp_location = sbol2.Range(uri=f"{comp.displayId}_location", start= len(composite_sequence_str)+fusion_site_length, end=len(composite_sequence_str)+len(part_extract_sequence[:-4])) #TODO check if seq len is correct
                    comp_anno = sbol2.SequenceAnnotation(uri=f"{comp.displayId}_annotation")
                    comp_anno.locations.add(comp_location)
                    anno_list.append(comp_anno)

            part_extract_definitions.extend(temp_extract_components)

            composite_sequence_str = composite_sequence_str + part_extract_sequence[:-fusion_site_length] #needs a version for linear
            composite_name = composite_name + '_' + re.match(r'^[^_]*_[^_]*', part_extract.displayId).group()

        for anno in anno_list:
            for location in anno.locations:
                print(anno.identity, location.start, location.end)
            

        # create dna component and sequence
        composite_component_definition, composite_seq = dna_componentdefinition_with_sequence2(f'composite_{composite_number}{composite_name}', composite_sequence_str)
        composite_component_definition.name = f'composite_{composite_number}{composite_name}'
        composite_component_definition.addRole('http://identifiers.org/SO:0000804') #engineered region

        part_extract_components = [] 
        

        for i, definition in enumerate(part_extract_definitions):
            def_object = document.getComponentDefinition(definition)
            comp = sbol2.Component(uri=def_object.displayId)
            comp.definition = definition
            composite_component_definition.components.add(comp)
            
            print(def_object.displayId)
            anno_list[i].component = comp           

        composite_component_definition.sequenceAnnotations = anno_list

        # for i in range(len(composite_component_definition.sequenceAnnotations)-1):
        #     # use precedes sequenceconstraint instead of v3#meets
        #     constraint = sbol2.sequenceconstraint.SequenceConstraint(uri=f'{composite_component_definition.sequenceAnnotations[i].displayId}_precedes_{composite_component_definition.sequenceAnnotations[i+1].displayId}', restriction=sbol2.SBOL_RESTRICTION_PRECEDES) #TODO: change!!!
        #     constraint.subject = composite_component_definition.sequenceAnnotations[i]
        #     constraint.object = composite_component_definition.sequenceAnnotations[i+1]
            
        #     composite_component_definition.sequenceConstraints.add(constraint)

        products_list.append([composite_component_definition, composite_seq])
        print(f"iteration {composite_number} with {composite} complete")
        composite_number += 1
    return products_list

## Tests

In [185]:
ligation_doc = sbol2.Document()
temp_doc = sbol2.Document()
reactants_list = []
assembly_plan = sbol2.ModuleDefinition('bb_assembly_plan')
parts = ['Test/pro_in_bb.xml','Test/rbs_in_bb.xml', 'Test/cds_in_bb.xml', 'Test/terminator_in_bb.xml']


temp_doc.read('Test/backbone.xml')
# run digestion, extract component + sequence, add to ligation_doc, reactants_list
md = temp_doc.getModuleDefinition('https://sbolcanvas.org/module1')
extracts_tuple_list, assembly_plan = backbone_digestion(md, [bsai], assembly_plan, temp_doc)
for extract, seq in extracts_tuple_list:
    try:
        ligation_doc.add(extract)
        ligation_doc.add(seq)
    except Exception as e: print(e)
    print(f"adding part {extract}")

ligation_doc.add(assembly_plan)
print(f"reactant 0: {extracts_tuple_list[0][0]}")
reactants_list.append(extracts_tuple_list[0][0])

for i, part in enumerate(parts):
    temp_doc.read(part)
    md = temp_doc.getModuleDefinition('https://sbolcanvas.org/module1')
    extracts_tuple_list, assembly_plan = part_digestion(md, [bsai], assembly_plan, temp_doc)

    for extract, sequence in extracts_tuple_list:
        try:
            ligation_doc.add(extract)
            ligation_doc.add(sequence)
        except Exception as e: print(e)
        print(f"doc {i}: adding part {extract}")

    reactants_list.append(extracts_tuple_list[0][0])
    
ligation_doc.add(bsai)
ligation_doc.write('ligation_test1.xml')

adding part http://examples.org/ComponentDefinition/UJHDBOTD_extracted_backbone/1
adding part http://examples.org/ComponentDefinition/UJHDBOTD_159_five_prime_oh/1
adding part https://sbolcanvas.org/Scar_F/1
adding part https://sbolcanvas.org/Cir_qxow/1
adding part http://examples.org/ComponentDefinition/UJHDBOTD_159_three_prime_oh/1
adding part https://sbolcanvas.org/Scar_A/1
reactant 0: http://examples.org/ComponentDefinition/UJHDBOTD_extracted_backbone/1
doc 0: adding part http://examples.org/ComponentDefinition/UJHDBOTD_70_extracted_part/1
doc 0: adding part http://examples.org/ComponentDefinition/UJHDBOTD_70_three_prime_oh/1
doc 0: adding part https://sbolcanvas.org/Scar_B/1
doc 0: adding part https://sbolcanvas.org/J23101/1
doc 0: adding part http://examples.org/ComponentDefinition/UJHDBOTD_70_five_prime_oh/1
(<SBOLErrorCode.SBOL_ERROR_URI_NOT_UNIQUE: 17>, 'Cannot add https://sbolcanvas.org/Scar_A/1 to Document. An object with this identity is already contained in the Document')
d

'Valid.'

In [186]:
for obj in ligation_doc:
    print(obj)

obj = ligation_doc.find('http://examples.org/ModuleDefinition/bb_assembly_plan/1')
print(type(obj))
for comp in obj.functionalComponents:
    print(comp.identity)

http://examples.org/ComponentDefinition/UJHDBOTD_extracted_backbone/1
http://examples.org/Sequence/UJHDBOTD_extracted_backbone_seq/1
http://examples.org/ComponentDefinition/UJHDBOTD_159_five_prime_oh/1
http://examples.org/Sequence/UJHDBOTD_159_five_prime_oh_sequence/1
https://sbolcanvas.org/Scar_F/1
https://sbolcanvas.org/Scar_F_sequence
https://sbolcanvas.org/Cir_qxow/1
https://sbolcanvas.org/Cir_qxow_sequence
http://examples.org/ComponentDefinition/UJHDBOTD_159_three_prime_oh/1
http://examples.org/Sequence/UJHDBOTD_159_three_prime_oh_sequence/1
https://sbolcanvas.org/Scar_A/1
https://sbolcanvas.org/Scar_A_sequence
http://examples.org/ModuleDefinition/bb_assembly_plan/1
http://examples.org/ComponentDefinition/UJHDBOTD_70_extracted_part/1
http://examples.org/Sequence/UJHDBOTD_70_extracted_part_seq/1
http://examples.org/ComponentDefinition/UJHDBOTD_70_three_prime_oh/1
http://examples.org/Sequence/UJHDBOTD_70_three_prime_oh_sequence/1
https://sbolcanvas.org/Scar_B/1
https://sbolcanvas.or

### Ligation Test

In [187]:
for definition in reactants_list:
    for anno in definition.sequenceAnnotations:
        for location in anno.locations:
            print(anno.identity, location.start, location.end)

http://examples.org/ComponentDefinition/UJHDBOTD_extracted_backbone/three_prime_overhang/1 2200 2203
http://examples.org/ComponentDefinition/UJHDBOTD_extracted_backbone/five_prime_overhang/1 1 4
http://examples.org/ComponentDefinition/UJHDBOTD_extracted_backbone/UJHDBOTD_159_backbone/1 5 2199
http://examples.org/ComponentDefinition/UJHDBOTD_70_extracted_part/three_prime_overhang/1 40 43
http://examples.org/ComponentDefinition/UJHDBOTD_70_extracted_part/five_prime_overhang/1 1 4
http://examples.org/ComponentDefinition/UJHDBOTD_70_extracted_part/UJHDBOTD_70_part/1 5 39
http://examples.org/ComponentDefinition/UJHDBOTD_45_extracted_part/three_prime_overhang/1 26 29
http://examples.org/ComponentDefinition/UJHDBOTD_45_extracted_part/five_prime_overhang/1 1 4
http://examples.org/ComponentDefinition/UJHDBOTD_45_extracted_part/UJHDBOTD_45_part/1 5 25
http://examples.org/ComponentDefinition/UJHDBOTD_106_extracted_part/three_prime_overhang/1 722 725
http://examples.org/ComponentDefinition/UJHDBOT

### WRITE TEST

In [188]:
print(f"params = {reactants_list}, {assembly_plan}, {type(ligation_doc)}")
pl = ligation2(reactants_list, assembly_plan, ligation_doc)

for l in pl:
    for obj in l:
        print(obj.identity)
        ligation_doc.add(obj)

ligation_doc.write('ligation_test_forreal.xml')

params = [<sbol2.componentdefinition.ComponentDefinition object at 0x127b25ee0>, <sbol2.componentdefinition.ComponentDefinition object at 0x13133c0e0>, <sbol2.componentdefinition.ComponentDefinition object at 0x133c37f50>, <sbol2.componentdefinition.ComponentDefinition object at 0x131735bb0>, <sbol2.componentdefinition.ComponentDefinition object at 0x127e319a0>], http://examples.org/ModuleDefinition/bb_assembly_plan/1, <class 'sbol2.document.Document'>
list composites per assembly [[<sbol2.componentdefinition.ComponentDefinition object at 0x127e319a0>, <sbol2.componentdefinition.ComponentDefinition object at 0x127b25ee0>, <sbol2.componentdefinition.ComponentDefinition object at 0x13133c0e0>, <sbol2.componentdefinition.ComponentDefinition object at 0x133c37f50>, <sbol2.componentdefinition.ComponentDefinition object at 0x131735bb0>]]
3p: http://examples.org/ComponentDefinition/UJHDBOTD_106_three_prime_oh/1
http://examples.org/SequenceAnnotation/Scar_A_annotation/1 1 4
http://examples.org

'Valid.'